# Create Azure SQL Database

This covers the multi-step process of creating an Azure SQL Database that can be queried by Power BI.  This invovles creating a server, a database, and updating the server firewall to allow azure access.   While this is one path to do this there are hundreds of permutations of how this can be done.  For example:

1. Does the resource group already exist?
1. When can we assume the server already exists?  The server is really just a logical placeholder and makes little impact on performance because the performance attributes are set at the database level.  (But what about elastic pools?)
1. If the server already exists can we assume the firewall is already set?
1. What about Active Directory permissions, does it make sense to have the managed identity pre-created and assigned Entra directory reader permissions?
1. Do we need high availability settings on the database?
1. Do we need to deploy databases in different azure regions?

This script is written to assume the following has already been done:

1. The logical server has already been created.
1. The server's managed identity has been created (user assigned or system managed)
1. The managed identity has been granted the "Directory Reader" role in Entra enabling us to add entra users to the SQL Database.
1. The server firewall has already been set on the database to allow Azure access.
1. The service principal has been set as the entra server admin.

Also, unlike some of our earlier examples, we're using the azure-mgmt-sql libraries to simplify the creation, we're doing this becasue they actually exist in this case.  This results in a few things that are a departure from other notebooks in this repo:

1. We have no need to create the header for the rest api call.
1. We're actually not using the raw AAD token now, but rather a credential object.  The python packages will handle getting a token for the appropriate scope under the covers.
1. We're no longer concerned with what the path to the API is, and whether the method is a put, post, patch or delete, this is abstracted from us.

For reference here are some sources:

1. Create Server - https://learn.microsoft.com/en-us/rest/api/sql/servers/update?view=rest-sql-2021-11-01&tabs=HTTP
1. Create Database - https://learn.microsoft.com/en-us/rest/api/sql/databases?view=rest-sql-2021-11-01
1. Firewall Rules - https://learn.microsoft.com/en-us/rest/api/sql/firewall-rules?view=rest-sql-2021-11-01
1. SqlManagementClient doc - https://learn.microsoft.com/en-us/python/api/azure-mgmt-sql/azure.mgmt.sql.sqlmanagementclient?view=azure-python 
1. Python samples of using azure-mgmt-sql:  https://github.com/Azure-Samples/azure-samples-python-management/tree/main/samples/sql

In [ ]:
pip install azure-mgmt-sql requests msal pyodbc

In [ ]:
resourceGroup = "cgmpbie"
subscriptionId = "7258b7d4-3429-4998-815b-8cd6954b7ef9"
serverName = "cgmpbiesqlserver"
databaseRoot = "cgmpbietenant"
location = "westus2"

#Add another Admin to the database for convenience so you can connect to the database in Azure Data Studio or Query Editor.
secondaryAdmin = "chmitch@microsoftanalyitcs.info"

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login.
#Note: for most of the other notebooks we're not passing in a scope, this is because the code in aadservice.property
#defaults the scope to a scope for the Power BI API.   Since we're calling the azure management API we need a different
#scope.
from services.aadservice import AadService
scope = 'https://management.azure.com/.default'
#aad_token = AadService.get_access_token(scope)
cred = AadService.get_credential()

#with the credential object, get the token for the azure management scope.
aadToken = cred.get_token(scope).token
headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadToken}

In [ ]:
#This code is entriely about getting a list of databases currently on the server (ie. "Tenant databases"), and
#incrementing a counter to create a new tenant database without a name confict.
from azure.mgmt.sql import SqlManagementClient
import json
from services.env import const

try:
    #Connect to the Azure Management Rest API.
    sqlClient = SqlManagementClient(credential=cred, subscription_id=const.subscriptionId)
    
    # Create database
    databases = sqlClient.databases.list_by_server(
        const.resourceGroup,
        const.serverName
    )

    keyPosition = 0
    keyValue = 0
    for database in databases:
        nameParts = database.name.split("_")
        #if there was an underscroe in the title, get the suffix of the database name.
        if len(nameParts) > 1:
            #if the suffix is greater than our max capture it as the new max
            if int(nameParts[1]) > keyValue:
                keyValue = int(nameParts[1])
    
    #Increment the Tenant Id for the new suffix
    keyValue = keyValue + 1 
    databaseName = f"{const.databaseRoot}_{keyValue}"
    
except KeyError:
    print(f"Database {databaseName} create failed")

In [ ]:
from azure.mgmt.sql import SqlManagementClient
import json

try:
    #
    sqlClient = SqlManagementClient(credential=cred, subscription_id=subscriptionId)
    
    # Create database
    database = sqlClient.databases.begin_create_or_update(
        const.resourceGroup,
        const.serverName,
        databaseName,
        {
            "location": const.location,
            "sku": {
                "name": "S0",
                "tier": "Standard"
            }
        }
        ).result()

    print(f"Database Created:\n{database}")
except KeyError:
    print(f"Database {databaseName} create failed")


Now that we've created a database on the precreated server, we want to do some post configuration on that database.  The server's admin account is running as a service principal.  In order to let us connect to that datbabase directly we need to add an additional Entra user to the database.  I'll do this using and ODBC connection and a couple SQL commands to create a local database user that corresponds to an Entra user and also grant that user owner permissions on the database.

This is an example of where you'd likely create a tenant specific user for the database and grant that user access.

Supporting docs:

1. How to install SQL ODBC drivers on Linux:  https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline
2. How to connect to a database with pyodbc drivers and a service principal login:  https://learn.microsoft.com/en-us/azure/azure-sql/database/azure-sql-passwordless-migration-python?view=azuresql&tabs=sign-in-azure-cli%2Cazure-portal-create%2Cazure-portal-assign%2Capp-service-identity

In [ ]:
databaseName="template"
serverName="cgmpbiesqlserver"

In [ ]:
import struct
import pyodbc
import json
from services.secretservice import SecretService
from services.env import const

dbUser = const.globalAdminUser
serverFqdn = f'{const.serverName}.database.windows.net'
driver = '{ODBC Driver 18 for SQL Server}'

#Get a credential for database access.
tokenBytes = cred.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(tokenBytes)}s', len(tokenBytes), tokenBytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256

#open the connection
conn_str = f'DRIVER={driver};SERVER={serverFqdn};DATABASE={databaseName};'
conn = pyodbc.connect(conn_str,attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})

#Create an external user on the databse and grant them DBO Access
sql = f"CREATE USER [{dbUser}] FROM EXTERNAL PROVIDER;"
conn.execute(sql)
sql = f"ALTER ROLE db_owner ADD MEMBER [{dbUser}];"
conn.execute(sql)

#This is important, the connection doesn't auto commit
conn.commit()